In [30]:
import ant_colony as ac
import json
import networkx as nx
import numpy as np
import pandas as pd

from scipy.spatial import distance_matrix

In [31]:
## parametros del algoritmo
data_path = '../practica-1-segunda-parte-ltejadal/datasets/ch71009.tsp'
init_node_ex = 0
hp_trials = 2
save_hp = True
seed = 19519159
n_cities = 30

In [32]:
with open(data_path) as f:
    # lectura de datos
    node_coord_start = None
    dimension = None
    lines = f.readlines()

In [33]:
# Información del archivo TSP
## Para la lectura de datos seguimos lo implementado por: https://github.com/DiegoVicen/som-tsp
i = 0
while not dimension or not node_coord_start:
    line = lines[i]
    if line.startswith('DIMENSION :'):
        dimension = int(line.split()[-1])
    if line.startswith('NODE_COORD_SECTION'):
        node_coord_start = i
    i = i+1
    
print('Problema con {} ciudades.'.format(dimension))

Problema con 71009 ciudades.


In [34]:
## Para la lectura de datos seguimos lo implementado por: https://github.com/DiegoVicen/som-tsp
ciudades_df = pd.read_csv(
            data_path,
            skiprows=node_coord_start + 1,
            sep=' ',
            names=['city', 'lat', 'lon'],
            dtype={'city': str, 'lat': np.float64, 'lon': np.float64},
            header=None,
            nrows=dimension)

# limpiar latitud y longitud (decimales)
ciudades_df['lat'] = ciudades_df['lat'].div(1000)
ciudades_df['lon'] = ciudades_df['lon'].div(1000)

In [35]:
sample_df = ciudades_df.sample(n_cities, random_state=seed)

In [36]:
## Calcular distancias en KMS
array_coord = sample_df[['lat','lon']].to_numpy()
d_mat = distance_matrix(array_coord, array_coord)*6373

In [37]:
G = nx.from_numpy_matrix(d_mat) 

In [ ]:
# graficar el graph
ac.plot_graph(G, m_plot='graph')

In [ ]:
study = ac.optim_h_params(G,
                          init_node = init_node_ex,
                          trials = hp_trials, 
                          save = save_hp)

In [ ]:
best_params = ac.load_params('best_hiper_params.db')

In [12]:
colony_ = ac.colony(G, init_node_ex, **best_params)

In [13]:
colony_.solve_tsp()

In [ ]:
colony_.plot_route(plt_size=(40, 40))

In [15]:
sln_r = colony_.best_route

In [16]:
sln_d = colony_.best_dist

In [24]:
sln_dic = {'route': sln_r, 'distance': float(sln_d)}

In [21]:
with open("aco_sln.json", "w") as outfile: 
    json.dump(sln_dic, outfile)
outfile.close()